<a href="https://colab.research.google.com/github/Sorrellee/Data-mining-technologies/blob/main/%D0%A2%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Разработка модели предсказания реакции на маркетинговые кампании на основе анализа социальных данных и поведения потребителей

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive

In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
# Загрузка данных
df = pd.read_csv('/content/gdrive/MyDrive/Лабораторные иск. инт./1/Marketingcampaigns.csv')

In [36]:
# Определение признаков и целевой переменной
features = ['Email Opened', 'Email Clicked', 'Discount offered']
target_variable = 'Purchased'
X = df[features]
y = df[target_variable]

In [37]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
# Стандартизация признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
# Подбор параметров с помощью Grid Search для RandomForestClassifier
param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [None, 10, 20, 30]}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

In [40]:
# Кросс-валидированная точность
scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
print(f'Кросс-валидированная точность: {scores.mean()}')

Кросс-валидированная точность: 0.2333333333333333


In [41]:
# Точность на тестовой выборке
print(f'Точность на тестовой выборке: {best_model.score(X_test, y_test)}')

Точность на тестовой выборке: 0.75


In [42]:
# Создание DataFrame с фактическими и предсказанными метками
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': best_model.predict(X_test)})

# Сохранение DataFrame в CSV-файл
results_df.to_csv('/content/gdrive/MyDrive/Лабораторные иск. инт./1/model_results.csv', index=False)